In [1]:
import pandas

In [2]:
import numpy

In [4]:
import numpy as np

def notDefined(variable):
    return variable is None

def checkfields(params, *fields):
    for field in fields:
        if field not in params:
            return False
    return True

def shift(seq, n):
    return seq[n:] + seq[:n]

def rmfilter_binary(images, display):
    # Implementation of binary filter (replace with actual filter logic)
    pass

def subLoadImages(params, id):
    print(f"[subLoadImages]: Loading images: {params['stim'][id]['imFile']}...")
    print(f"[subLoadImages]: Loading images for scan {id}...")
    
    imFile = params['stim'][id]['imFile']
    if not checkfields(params, 'stim', 'imFile'):
        raise ValueError("Need the image matrix saved from scan")
    
    if not imFile.endswith('.mat'):
        imFile = f"Stimuli/{imFile}"
    
    I = np.load(imFile)
    # TODO: make compatible with RGB images (now assumes grayscale)
    
    paramsFile = params['stim'][id]['paramsFile']
    if not checkfields(params, 'stim', 'paramsFile'):
        raise ValueError("Need the experiment params file from scan")
    
    if not paramsFile.endswith('.mat'):
        paramsFile = f"Stimuli/{paramsFile}"
    
    P = np.load(paramsFile)
    return P, I

def subFilterImages(I, P, params, id):
    print("[subFilterImages]: Filtering images...")
    if not checkfields(params, 'stim', 'imFilter'):
        params['stim'][id]['imFilter'] = 'binary'
    
    # try to determine colormap parameters
    try:
        display = P['params']['display']
    except KeyError:
        display = None
        print("[subFilterImages]: No calibration file found. This may affect image filtering.")
    
    # Apply filter (replace with actual filter function)
    I['images'] = rmfilter_binary(I['images'], display)
    return I

def subSamplingGrid(params):
    print("[subSamplingGrid]: Creating sampling grid...")
    nSamples = params['analysis']['numberStimulusGridPoints']
    fieldSize = params['analysis']['fieldSize']
    sampleRate = params['analysis']['sampleRate']
    
    mygrid = np.arange(-fieldSize, fieldSize + sampleRate, sampleRate)
    x, y = np.meshgrid(mygrid, mygrid)
    
    params['analysis']['X'] = x.flatten()
    params['analysis']['Y'] = y.flatten()
    
    if len(params['analysis']['X']) != (1 + nSamples * 2) ** 2:
        raise ValueError("[subSamplingGrid]: error in grid creation")
    
    return x, y, params

def subSpatialDownsample(I, params):
    print("[subSpatialDownsample]: Downsampling images to stimulus grid...")
    nImages = I['images'].shape[2]
    nSamples = params['analysis']['numberStimulusGridPoints']
    resampled = np.zeros((len(params['analysis']['X']), nImages))
    
    for ii in range(nImages):
        tmp = np.array(Image.fromarray(I['images'][:, :, ii]).resize((1 + 2 * nSamples, 1 + 2 * nSamples), Image.NEAREST))
        resampled[:, ii] = tmp.flatten()
    
    I['images'] = resampled
    return I

def subTemporalDownsample(I, P, params, id):
    print("[subTemporalDownsample]: Averaging images within a TR...")
    framePeriod = params['stim'][id]['framePeriod']
    nFrames = params['stim'][id]['nFrames']
    
    if 'prescanDuration' in P['params']:
        params['stim'][id]['prescanDuration'] = P['params']['prescanDuration'] / framePeriod
    
    prescanDuration = params['stim'][id]['prescanDuration']
    nFrames += prescanDuration
    seq = P['stimulus']['seq']
    seqTiming = P['stimulus']['seqtiming']
    images = np.zeros((len(params['analysis']['X']), nFrames))
    
    imOnset = seqTiming
    imOffset = shift(seqTiming, -1)
    imOffset[-1] = framePeriod * nFrames
    
    for f in range(nFrames):
        frameOnset = framePeriod * f
        frameOffset = framePeriod * (f + 1)
        imDur = np.minimum(imOffset, frameOffset) - np.maximum(imOnset, frameOnset)
        imDur[imDur < 0.001] = 0
        
        img = np.zeros_like(I['images'][:, 0])
        ii = np.where(imDur > 0)[0]
        for im in ii:
            img += imDur[im] * I['images'][:, seq[im]]
        img /= framePeriod
        
        if 'fliprotate' in params['stim'][id]:
            if params['stim'][id]['fliprotate'][0]:
                img = np.fliplr(img)
            try:
                if params['stim'][id]['fliprotate'][1]:
                    img = np.flipud(img)
                if params['stim'][id]['fliprotate'][2] != 0:
                    img = np.rot90(img, params['stim'][id]['fliprotate'][2])
            except IndexError:
                pass
        
        images[:, f] = img
    
    return images, params

# Example usage:
params = {
    'stim': {
        1: {'imFile': 'Stimuli/bar_stimulus_masks_1300ms_images.mat', 'paramsFile': 'Stimuli/bar_stimulus_masks_1300ms_params.mat', 'framePeriod': 2, 'nFrames': 100, 'prescanDuration': 10}
    },
    'analysis': {
        'numberStimulusGridPoints': 5,
        'fieldSize': 10,
        'sampleRate': 1
    }
}

id = 1
P, I = subLoadImages(params, id)
I = subFilterImages(I, P, params, id)
x, y, params = subSamplingGrid(params)
I = subSpatialDownsample(I, params)
images, params = subTemporalDownsample(I, P, params, id)

print(f"[Main]: Done. Save the images and return.")
print(f"params['stim'][{id}]['images']: {images}")


[subLoadImages]: Loading images: Stimuli/bar_stimulus_masks_1300ms_images.mat...
[subLoadImages]: Loading images for scan 1...


ValueError: Need the image matrix saved from scan

In [5]:
print(params)
P, I = subLoadImages(params, id)


{'stim': {1: {'imFile': 'Stimuli/bar_stimulus_masks_1300ms_images.mat', 'paramsFile': 'Stimuli/bar_stimulus_masks_1300ms_params.mat', 'framePeriod': 2, 'nFrames': 100, 'prescanDuration': 10}}, 'analysis': {'numberStimulusGridPoints': 5, 'fieldSize': 10, 'sampleRate': 1}}
[subLoadImages]: Loading images: Stimuli/bar_stimulus_masks_1300ms_images.mat...
[subLoadImages]: Loading images for scan 1...


ValueError: Need the image matrix saved from scan

In [7]:
def subLoadImages(imFile, paramsFile):
    # Load image matrix from imFile allowing pickled data
    I = np.load(imFile, allow_pickle=True)
    # Load params from paramsFile allowing pickled data
    P = np.load(paramsFile, allow_pickle=True)
    return P, I

# Define other functions here (subFilterImages, subSamplingGrid, subSpatialDownsample, subTemporalDownsample, etc.)

# Example usage:

# File paths
imFile = 'Stimuli/bar_stimulus_masks_1300ms_images.mat'
paramsFile = 'Stimuli/bar_stimulus_masks_1300ms_params.mat'

# Load images and parameters
P, I = subLoadImages(imFile, paramsFile)

UnpicklingError: Failed to interpret file 'Stimuli/bar_stimulus_masks_1300ms_images.mat' as a pickle

In [17]:
from scipy.io import loadmat

# Load .mat file
img_data = loadmat('Stimuli/bar_stimulus_masks_1300ms_images.mat')

In [18]:
variable_names = img_data.keys()

# Print variable names
for variable_name in variable_names:
    print(f"Variable Name: {variable_name}")

Variable Name: __header__
Variable Name: __version__
Variable Name: __globals__
Variable Name: images


In [19]:
I = img_data["images"]

In [20]:
params_data = loadmat('Stimuli/bar_stimulus_masks_1300ms_params.mat')

In [22]:
variable_names = params_data.keys()

# Print variable names
for variable_name in variable_names:
    print(f"Variable Name: {variable_name}")

Variable Name: __header__
Variable Name: __version__
Variable Name: __globals__
Variable Name: stimulus


In [23]:
P = params_data["stimulus"]

In [30]:
P.shape

(1, 1)

In [25]:
def subFilterImages(I, P, params, id):
    print("[subFilterImages]: Filtering images...")
    if not checkfields(params, 'stim', 'imFilter'):
        params['stim'][id]['imFilter'] = 'binary'
    
    # try to determine colormap parameters
    try:
        display = P['params']['display']
    except KeyError:
        display = None
        print("[subFilterImages]: No calibration file found. This may affect image filtering.")
    
    # Apply filter (replace with actual filter function)
    I['images'] = rmfilter_binary(I['images'], display)
    return I

In [27]:
params = {
    'stim': {
        1: {'imFile': 'Stimuli/bar_stimulus_masks_1300ms_images.mat', 'paramsFile': 'Stimuli/bar_stimulus_masks_1300ms_params.mat', 'framePeriod': 2, 'nFrames': 100, 'prescanDuration': 10}
    },
    'analysis': {
        'numberStimulusGridPoints': 5,
        'fieldSize': 10,
        'sampleRate': 1
    }
}

In [28]:
I = subFilterImages(I, P, params, id)

[subFilterImages]: Filtering images...


ValueError: no field of name params

In [31]:
P.keys()

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [35]:
import matplotlib.pyplot as plt

# Assuming 'images' variable contains the processed image data
# For example, you can use the provided code to generate 'images'

# Visualize a single frame (change index to visualize different frames)
frame_index = 0

# Reshape the flattened image data to its original grid shape
n_samples = int(np.sqrt(len(images)))
image_grid = images[:, frame_index].reshape((n_samples, n_samples))

# Plot the image
plt.figure(figsize=(8, 6))
plt.imshow(image_grid, cmap='gray')  # Use cmap='viridis' for a different color map
plt.colorbar()
plt.title(f"Processed Image - Frame {frame_index + 1}")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

NameError: name 'images' is not defined